# More features

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from oemof.solph import (Sink, Source, Transformer, Bus, Flow, Model,
                         EnergySystem, Investment, NonConvex)
import oemof.outputlib as outputlib
import oemof.solph as solph
import numpy as np
import holoviews as hv
hv.extension('bokeh')
import xarray as xr
import pyomo.environ as po
import time
solver = 'cbc'

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [8.0, 6.0]

In [ ]:
color_dict ={
             'coal': '#755d5d',
             'gas': '#c76c56',
             'oil': '#494a19',
             'lignite': '#56201d',
             'wind': '#4ca7c3',
             'pv': '#ffde32',
             'excess_el': '#9a9da1',
             'pp_coal': '#755d5d',
             'pp_gas': '#c76c56',
             'pp_chp': '#eeac7e',
             'b_heat_source': '#cd3333',
             'heat_source': '#cd3333',
             'heat_pump': '#42c77a',
             'electricity': '#0079ff',
             'demand_el': '#0079ff',
             'shortage_el': '#ff2626',
             'excess_el': '#ff2626',
             'biomass': '#01b42e',
             'pp_biomass': '#01b42e'}

In [ ]:
datetimeindex = pd.date_range('1/1/2016', periods=24*10, freq='H')
filename = 'input_data.csv'
data = pd.read_csv(filename, sep=",")


def run_basic_energysystem(n_val_wind, n_val_solar):
    start = time.time()
    # initialize and provide data
    energysystem = EnergySystem(timeindex=datetimeindex)

    # buses
    bcoal = Bus(label='coal', balanced=False)
    bgas = Bus(label='gas', balanced=False)
    bel = Bus(label='electricity')
    energysystem.add(bcoal, bgas, bel)

    # sources
    energysystem.add(Source(label='wind', outputs={bel: Flow(
        actual_value=data['wind'], nominal_value=n_val_wind, fixed=True)}))

    energysystem.add(Source(label='pv', outputs={bel: Flow(
        actual_value=data['pv'], nominal_value=n_val_solar, fixed=True)}))

    # excess and shortage to avoid infeasibilies
    energysystem.add(Sink(label='excess_el', inputs={bel: Flow()}))
    energysystem.add(Source(label='shortage_el',
                         outputs={bel: Flow(variable_costs=200)}))

    # demands (electricity/heat)
    energysystem.add(Sink(label='demand_el', inputs={bel: Flow(
        nominal_value=65, actual_value=data['demand_el'], fixed=True)}))

    # power plants
    energysystem.add(Transformer(
        label='pp_coal',
        inputs={bcoal: Flow()},
        outputs={bel: Flow(nominal_value=20.2, variable_costs=25)},
        conversion_factors={bel: 0.39}))

    energysystem.add(Transformer(
        label='pp_gas',
        inputs={bgas: Flow()},
        outputs={bel: Flow(nominal_value=41, variable_costs=40)},
        conversion_factors={bel: 0.50}))

    # create optimization model based on energy_system
    optimization_model = Model(energysystem=energysystem)

    # solve problem
    optimization_model.solve(solver=solver,
                             solve_kwargs={'tee': False, 'keepfiles': False})

    results = outputlib.processing.results(optimization_model)

    results_el = outputlib.views.node(results, 'electricity')
    el_sequences = results_el['sequences']
    el_prod = el_sequences[[(('wind', 'electricity'), 'flow'),
                            (('pv', 'electricity'), 'flow'),
                            (('pp_coal', 'electricity'), 'flow'),
                            (('pp_gas', 'electricity'), 'flow'),
                            (('shortage_el', 'electricity'), 'flow')]]
    
    inputs = outputlib.processing.convert_keys_to_strings(outputlib.processing.parameter_as_dict(optimization_model))
    nom_vals = [[key, value['scalars']['nominal_value']] for key, value in inputs.items() if 'nominal_value' in value['scalars']]
    nom_vals = pd.DataFrame(nom_vals, columns=['flow', 'nominal_value'])
    summed_flows = [(key, value['sequences'].sum()[0]) for key, value in outputlib.processing.convert_keys_to_strings(results).items()]
    summed_flows = pd.DataFrame(summed_flows, columns=['flow', 'summed_flows'])

    end = time.time()
    print('simulation lasted: ', end - start, 'sec')
    
    return el_sequences[(('electricity', 'demand_el'), 'flow')], el_prod, nom_vals, summed_flows

In [ ]:
n_vals_wind = range(30,70,10)
n_vals_solar = range(10,70,10)
list_i = []
for i in n_vals_wind:
    list_j = []
    for j in n_vals_solar:
        el_demand, el_prod, nom_vals, summed_flows = run_basic_energysystem(i, j)
        el_prod_indexed = np.array(el_prod)
        list_j.append(el_prod_indexed)
    list_i.append(list_j)

In [ ]:
stacked_results = np.stack(list_i, axis=0)

In [ ]:
times = el_prod.index
producer = el_prod.columns
stacked_results = xr.DataArray(stacked_results, coords=[n_vals_wind, n_vals_solar, times, producer], dims=['n_vals_wind', 'n_vals_solar', 'times', 'producer'])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
for key in el_prod.keys():
    color_dict[key] = color_dict[key[0][0]] 
c=[color_dict.get(x, '#333333') for x in el_prod.columns]
el_prod.plot.area(ax=ax, color=c)
el_demand.plot(ax=ax, linewidth=3, c='k')
legend = ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)) # legend outside of plot

In [ ]:
def create_interactive_plot(data_dispatch, data_bars):
    """

    """    
    %%output size=150
    dispatch_hmap = hv.HoloMap({(i, k): hv.Area.stack(hv.Overlay([hv.Area(data_dispatch[k, i, :, j]) for j in range(n_stack)]))
                       for i in range(n_i) for k in range(n_k)}, kdims=['price pv', 'price wind']).options(width=500, height=200)

    bar_hmap = hv.HoloMap({(i, k): hv.Bars(data_bars, hv.Dimension('Technology'), 'Installed capacity')
                       for i in range(n_i) for k in range(n_k)}, kdims=['price pv', 'price wind'])

    layout = hv.Layout(dispatch_hmap + bar_hmap).cols(1)

    ## https://github.com/ioam/holoviews/issues/1819
    renderer = hv.renderer('bokeh')

    # Using renderer save
    # renderer.save(dispatch_hmap + bar_hmap, 'interactive_modeling')

    renderer.save(dispatch_hmap + bar_hmap, 'interactive_modeling')

n_stack = len(producer)
t = 240
n_i = len(n_vals_solar)
n_k = len(n_vals_wind)

# values = np.random.rand(n_stack, t, n_i, n_k)
data_dispatch = stacked_results
data_bars = [('nom_val_wind', 1),('nom_val_pv', 1)]

create_interactive_plot(data_dispatch, data_bars)

In [ ]:
data_dispatch.shape

In [ ]:
ax1 = summed_flows.plot.bar(x='flow', y='summed_flows', rot=0)
ax2 = nom_vals.plot.bar(x='flow', y='nominal_value', rot=0)